In [18]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import mysql.connector

In [2]:
df = pd.read_csv("../datos/attack_limpio.csv", index_col = 0)
df.columns

Index(['Unnamed: 0', 'case_number', 'year', 'type', 'country', 'area',
       'location', 'activity', 'name', 'age', 'time', 'species_', 'href',
       'injury', 'date', 'mes', 'fatal_', 'sex2', 'species_tipo', 'edad'],
      dtype='object')

In [5]:
df_clima = pd.read_csv('../datos/paises_meteo.csv', index_col=0)

In [6]:
df_clima.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'latitud', 'longitud', 'pais'],
      dtype='object')

In [30]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va a recibir dos parámetros que son fijos a lo largo de toda la BBDD, el nombre de la BBDD y 
        # la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="127.0.0.1",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
            
    # método para crear tablas  e insertar datos en ellas   
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="127.0.0.1",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    # método para comprobar si el pais que estamos intentando meter ya existe en la BBDD. 
    # En caso de que ya exista no será necesario que la insertemos de nuevo
    def check_pais (self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        # query para extraer los valores únicos de paises de la tabla de paises
        query_existe_pais = f"""
                SELECT DISTINCT pais FROM ataques
                """
        mycursor.execute(query_existe_pais)
        pais = mycursor.fetchall()
        return pais

    def sacar_id_pais(self, pais):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT id_pais FROM ataques WHERE pais = '{pais}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos ese pais en la BBDD y por lo tanto no te podemos dar su id. "
    

In [12]:
bbdd = 'tiburones'
contraseña = 'AlumnaAdalab'

In [31]:
cargar_tiburones = Cargar(bbdd, contraseña)

In [25]:
cargar_tiburones

In [32]:
cargar_tiburones.crear_bbdd

<bound method Cargar.crear_bbdd of <__main__.Cargar object at 0x7f1283128760>>

In [9]:
# tabla ataques

for indice, fila in df.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_ataques = f"""
                INSERT INTO ataques ('year', 'type', 'country', 'area',
       'location', 'activity', 'name', 'age', 'time', 'species_', 'href',
       'injury', 'date', 'mes', 'fatal_', 'sex2', 'species_tipo', 'edad') 
                VALUES ("{fila["year"]}", "{fila["type"]}", "{fila["country"]}","{fila["area"]}", "{fila["location"]}", "{fila["activity"]}",
                "{fila["name"]}", "{fila["age"]}", "{fila["time"]}", "{fila["species_"]}, "{fila["href"]}", "{fila["injury"]}",
                "{fila["date"]}", "{fila["mes"]}", "{fila["fatal_"]}", "{fila["sex2"]}", "{fila["species_tipo"]}", "{fila["edad"]}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    # pais = carga.check_pais()
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
 
    # if len(pais) == 0 or fila['pais'] not in pais[0]: 
        #carga.crear_insertar_tabla( query_ataques)

    #else:
     #   print(f"{fila['pais']} ya esta en nuestra BBDD")

In [ ]:
tabla_ataques = ''' CREATE TABLE IF NOT EXISTS `ataques` (
    `id_ataques` INT NOT NULL AUTO_INCREMENT, 
    `year` INT NOT NULL, 
    `type` VARCHAR(50) DEFAULT NULL, 
    `country` VARCHAR(50) DEFAULT NULL,  
    `age` FLOAT DEFAULT NULL,
    `species_` VARCHAR(50) DEFAULT NULL,
    `fecha_limpia` VARCHAR (10) DEFAULT NULL,
    `fatal` CHAR (1) DEFAULT NULL,
    `sex` CHAR (1) DEFAULT NULL,
    `activity` VARCHAR (250) DEFAULT NULL,
    PRIMARY KEY (`id_ataques`))
    
    ENGINE = InnoDB;
    '''

In [10]:
df_clima.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'latitud', 'longitud', 'pais'],
      dtype='object')

In [ ]:
def sacar_id_pais(contraseña, nombre_bbdd, pais):
    
    # creamos la conexion con el servidor y la BBDD
    cnx = mysql.connector.connect(user='root', password= f'{contraseña}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}")
    mycursor = cnx.cursor()

    # creamos la query para sacar el id de la pais que queramos
    query_sacar_id = f"SELECT id_pais FROM clima WHERE pais = '{pais}'"
    
    # puede que el id de la pais que estemos intentando insertar no este en nuestra BBDD. Por lo que vamos a hacer un try except para que no nos falle el código
    try: 
        mycursor.execute(query_sacar_id)
        id_ = mycursor.fetchall()[0][0]
        return id_
    
    except: 
        return "Sorry, no tenemos esa pais en la BBDD y por lo tanto no te podemos dar su id. "

In [ ]:
# tabla clima

for indice, fila in df.iterrows():

    id_pais = sacar_id_pais("AlumnaAdalab", "clima", fila['pais'])
    
    query_clima = f"""
                INSERT INTO clima (cloudcover, lifted_index, prec_type, prec_amount, temp2m, rh2m, weather, wind_direction, wind_speed, seeing, transparency, idtiempo, idlocalidad) 
                VALUES ({fila["cloudcover"]}, {fila['lifted_index']}, "{fila['prec_type']}", {fila['prec_amount']},{fila['temp2m']},{fila['rh2m']}, "{fila['weather']}","{fila['wind10m.direction']}",
                {fila['wind10m.speed']}, {fila['seeing']}, {fila['transparency']}, {id_pais});
                """
    # en vez de carga se pone el nombre de la variable que nosotros creemos cuando llamamos a la clase de crear tabla: carga.crear_insertar_tabla( query_medidas)
    